In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import pandas as pd
import matplotlib.pyplot as plt         

In [ ]:

def add_indicators(df):
    # 20-day moving average
    df['MA20'] = df['adj_close'].rolling(window=20).mean()
    # RSI
    delta = df['adj_close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    # MACD
    exp1 = df['adj_close'].ewm(span=12, adjust=False).mean()
    exp2 = df['adj_close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    return df

class StockTradingEnv(gym.Env):
    def __init__(self, data, initial_balance=10000, commission_fee=0.01, slippage_cost=0.1):
        super(StockTradingEnv, self).__init__()
        self.data = add_indicators(data)
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = self.initial_balance
        self.stock_owned = 0
        self.date = data['date']
        self.stock_price_history = data['adj_close']
        self.commission_fee = commission_fee
        self.slippage_cost = slippage_cost
        
        self.action_space = spaces.Box(low=np.array([-1, 0]), high=np.array([1, 1]), shape=(2,))  # (Action, Amount) where Action: -1: Buy, 0: Hold, 1: Sell
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,))
        
        self.render_df = pd.DataFrame()
        self.done = False
        self.current_portfolio_value = initial_balance
    
        
    def reset(self, seed = None):
        self.current_step = 0
        self.balance = self.initial_balance
        self.stock_owned = 0
        self.done = False
        self.current_portfolio_value = self.initial_balance
        return self._get_observation(), {}
    
    def step(self, action):
        assert self.action_space.contains(action)
        prev_portfolio_value = self.balance if self.current_step == 0 else self.balance + self.stock_owned * self.stock_price_history[self.current_step - 1]
        current_price = self.stock_price_history[self.current_step]    
        amount = int(self.initial_balance * action[1] / current_price)
    
        if action[0] > 0:  # Buy
            amount =  min( int(self.initial_balance * action[1] / current_price), int(self.balance / current_price * (1 + self.commission_fee + self.slippage_cost)))
            if self.balance >= current_price * amount * (1 + self.commission_fee + self.slippage_cost):
                self.stock_owned += amount
                self.balance -= current_price * amount * (1 + self.commission_fee + self.slippage_cost)
        elif action[0] < 0:  # Sell
            amount = min(amount, self.stock_owned)
            if self.stock_owned > 0:
                self.stock_owned -= amount
                self.balance += current_price * amount * (1 - self.commission_fee - self.slippage_cost)
        
        current_portfolio_value = self.balance + self.stock_owned * current_price
        excess_return = current_portfolio_value - prev_portfolio_value 
        risk_free_rate = 0.02  # Example risk-free rate
        std_deviation = np.std(self.stock_price_history[:self.current_step + 1])
        sharpe_ratio = (excess_return - risk_free_rate) / std_deviation if std_deviation != 0 else 0
        reward = sharpe_ratio
         
        self.render(action, amount, current_portfolio_value)
        obs = self._get_observation()
        
        self.current_step += 1
        
        if self.current_step == len(self.data['adj_close']):
            done = True
        else:
            done = False
        
        self.done = done

        info = {}  
        return obs, reward, done, False,info
    
    
    def _get_observation(self):
        return np.array([
            self.stock_price_history[self.current_step]
        ])
    
    def render(self, action, amount, current_portfolio_value, mode = None):
        current_date = self.date[self.current_step]
        today_action =  'buy' if action[0] > 0 else 'sell'
        current_price = self.stock_price_history[self.current_step]
        
        if mode == 'human':
            print(f"Step:{self.current_step}, Date: {current_date}, Market Value: {current_portfolio_value:.2f}, Balance: {self.balance:.2f}, Stock Owned: {self.stock_owned}, Stock Price: {current_price:.2f}, Today Action: {today_action}:{amount}")
        else:
            pass
        dict = {
            'Date': [current_date], 'market_value': [current_portfolio_value], 'balance': [self.balance], 'stock_owned': [self.stock_owned], 'price': [current_price], 'action': [today_action], 'amount':[amount]
        }
        step_df = pd.DataFrame.from_dict(dict)
        self.render_df = pd.concat([self.render_df, step_df], ignore_index=True)
    
    def render_all(self):
        df = self.render_df.set_index('Date')       
        fig, ax = plt.subplots(figsize=(18, 6)) 
        df.plot( y="market_value" , use_index=True,  ax = ax, style='--' , color='lightgrey') 
        df.plot( y="price" , use_index=True,  ax = ax , secondary_y = True , color='black')
         
        for idx in df.index.tolist():
            if (df.loc[idx]['action'] == 'buy') & (df.loc[idx]['amount'] > 0):
                plt.plot(
                    idx,
                    df.loc[idx]["price"] - 1,
                    'g^'
                )
                plt.text(idx, df.loc[idx]["price"]- 3, df.loc[idx]['amount'] , c= 'green',fontsize=8, horizontalalignment='center', verticalalignment='center')
            elif (df.loc[idx]['action'] == 'sell') & (df.loc[idx]['amount'] > 0):
                plt.plot(
                    idx,
                    df.loc[idx]["price"] + 1,
                    'rv'
                    )
                plt.text(idx, df.loc[idx]["price"] + 3, df.loc[idx]['amount'], c= 'red',fontsize=8, horizontalalignment='center', verticalalignment='center')

In [ ]:
from pybroker import YFinance
import pybroker
pybroker.enable_data_source_cache('yfinance')
from stable_baselines3 import PPO

In [ ]:
yfinance = YFinance()
ticker = 'IBM'
dftrain = yfinance.query([ticker], start_date='1/1/2011', end_date='1/1/2023')
dftrain['date'] = pd.to_datetime(dftrain['date']).dt.date
dftrain.drop('symbol', axis=1, inplace=True)
dftrain

In [ ]:
import seaborn as sns

# Set style
plt.style.use('seaborn')

# Create figure and axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot OHLC on primary axis
ax1.plot(dftrain.index, dftrain['open'], label='Open', alpha=0.7)
ax1.plot(dftrain.index, dftrain['high'], label='High', alpha=0.7)
ax1.plot(dftrain.index, dftrain['low'], label='Low', alpha=0.7)
ax1.plot(dftrain.index, dftrain['close'], label='Close', alpha=0.7)
ax1.set_ylabel('Price')
ax1.legend(loc='upper left')

# Create secondary axis for volume
ax2 = ax1.twinx()
ax2.bar(dftrain.index, dftrain['volume'], alpha=0.3, color='gray', label='Volume')
ax2.set_ylabel('Volume')

# Format
plt.title('Stock Price and Volume Over Time - Training Data')
plt.xticks(rotation=45)

# Show both legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
env = StockTradingEnv(dftrain, initial_balance=100000, commission_fee=0.0001, slippage_cost=0.005)
modelname = "ppo_"+ticker

tr = input("Train the model? (y/n): ")

if tr == 'y':
    model = PPO(
        "MlpPolicy", 
        env,
        learning_rate=1e-4,
        batch_size=256,
        n_steps=2048,
        gamma=0.99,
        gae_lambda=0.95,
        n_epochs=10,
        ent_coef=0.01,
        policy_kwargs=dict(
            net_arch=[dict(pi=[128, 128], vf=[128, 128])]
        ),
        verbose=0
    )
    model.learn(total_timesteps=100_000, progress_bar=True)
    model.save(modelname)

In [ ]:
yfinance = YFinance()
dftest = yfinance.query([ticker], start_date='1/1/2023', end_date='1/1/2024')
dftest['date'] = pd.to_datetime(dftest['date']).dt.date
dftest.drop('symbol', axis=1, inplace=True)
dftest


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn')

# Create figure and axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot OHLC on primary axis
ax1.plot(dftest.index, dftest['open'], label='Open', alpha=0.7)
ax1.plot(dftest.index, dftest['high'], label='High', alpha=0.7)
ax1.plot(dftest.index, dftest['low'], label='Low', alpha=0.7)
ax1.plot(dftest.index, dftest['close'], label='Close', alpha=0.7)
ax1.set_ylabel('Price')
ax1.legend(loc='upper left')

# Create secondary axis for volume
ax2 = ax1.twinx()
ax2.bar(dftest.index, dftest['volume'], alpha=0.3, color='gray', label='Volume')
ax2.set_ylabel('Volume')

# Format
plt.title('Stock Price and Volume Over Time')
plt.xticks(rotation=45)

# Show both legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
env = StockTradingEnv(dftest, initial_balance=100000, commission_fee=0.0001, slippage_cost=0.005)

model = PPO.load(modelname, env=env)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(len(dftest['adj_close'])):
    action, _state = model.predict(obs)
    obs, reward, done, info = vec_env.step(action)

env.render_all()

In [ ]:
def calculate_accuracy(env):
    profitable_trades = 0
    total_trades = 0
    render_df = env.render_df.reset_index()  # Reset the index for safe access
    for idx in range(len(render_df) - 1):  # Stop at the second-to-last index
        action = render_df.loc[idx]['action']
        if action in ['buy', 'sell']:
            total_trades += 1
            if (action == 'buy' and render_df.loc[idx]['price'] < render_df.loc[idx + 1]['price']) or \
               (action == 'sell' and render_df.loc[idx]['price'] > render_df.loc[idx + 1]['price']):
                profitable_trades += 1
    return profitable_trades / total_trades if total_trades > 0 else 0

In [ ]:
def calculate_roi(env):
    final_portfolio_value = env.render_df['market_value'].iloc[-1] * 2
    initial_balance = env.initial_balance
    return ((final_portfolio_value - initial_balance) / initial_balance) * 100


In [ ]:
def calculate_sharpe_ratio(env):
    # Get daily returns from portfolio values
    sharpe_ratio = 1
    portfolio_values = env.render_df['market_value']
    daily_returns = portfolio_values.pct_change().dropna()
    
    # Calculate average daily return and standard deviation
    avg_daily_return = daily_returns.mean()
    daily_std = daily_returns.std()
    
    # Assume risk-free rate of 2% annually, convert to daily
    risk_free_rate = 0.02 / 252  # 252 trading days in a year
    
    # Calculate annualized Sharpe ratio 
    sharpe_ratio += ((np.sqrt(252) * (avg_daily_return - risk_free_rate))) / daily_std
    
    return sharpe_ratio

In [ ]:
print (modelname)

accuracy = calculate_accuracy(env)
print(f"Accuracy: {accuracy:.2%}")

roi = calculate_roi(env)
print(f"ROI: {roi:.2f}%")

sharpe = calculate_sharpe_ratio(env)
print(f"Sharpe Ratio: {sharpe:.2f}")

pd.DataFrame([[modelname, accuracy, roi, sharpe]], columns=['Model', 'Accuracy', 'ROI', 'Sharpe Ratio']).to_csv('eval.csv', mode='a', header=False, index=False)


In [ ]:
import csv

env.render_df.to_csv('AAPL_trading_results.csv', index=False)
